<a href="https://colab.research.google.com/github/Aksa21/ddos_detection_uav/blob/main/dataextraction_ddos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444328 sha256=c115df7a8062221ded606d49928b3a6db20b7a7f75aeba2ab2b70e4f61118bb9
  Stored in directory: /root/.cache/pip/wheels/82/b7/03/8344d8cf6695624746311bc0d389e9d05535ca83c35f90241d
Successfully built scapy


In [ ]:
from scapy.all import *
import csv

def pcap_to_csv(pcap_filename, csv_filename):
    packets = rdpcap(pcap_filename)

    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ['timestamp', 'source_ip', 'destination_ip', 'source_port', 'destination_port', 'protocol', 'packet_size']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()

        for pkt in packets:
            timestamp = pkt.time
            source_ip = pkt[IP].src if IP in pkt else None
            destination_ip = pkt[IP].dst if IP in pkt else None
            source_port = pkt[IP].sport if IP in pkt and hasattr(pkt[IP], 'sport') else None
            destination_port = pkt[IP].dport if IP in pkt and hasattr(pkt[IP], 'dport') else None
            protocol = pkt[IP].proto if IP in pkt else None
            packet_size = len(pkt)

            writer.writerow({
                'timestamp': timestamp,
                'source_ip': source_ip,
                'destination_ip': destination_ip,
                'source_port': source_port,
                'destination_port': destination_port,
                'protocol': protocol,
                'packet_size': packet_size
            })

# Specify input PCAP file and output CSV file
input_pcap_filename = '/content/25dataaa.pcap'
output_csv_filename = 'norm_gold.csv'

pcap_to_csv(input_pcap_filename, output_csv_filename)
print("PCAP to CSV conversion complete. Results written to", output_csv_filename)



PCAP to CSV conversion complete. Results written to norm_gold.csv


In [ ]:
from glob import glob
import pandas as pd

code for packet_info_ddos

In [ ]:
import pandas as pd

# Sample data with Unix epoch timestamps
data = pd.read_csv('/content/norm_gold.csv')

# Convert Unix epoch timestamps to pandas Timestamp objects
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')

# Find the minimum Unix epoch timestamp
min_timestamp = data['timestamp'].min()

# Subtract the minimum timestamp to shift the time scale
data['adjusted_timestamp'] = (data['timestamp'] - min_timestamp) + pd.Timedelta(seconds=1)

print(data)


                           timestamp      source_ip  destination_ip  \
0      2023-08-25 09:52:04.857777152    10.42.0.111      10.42.0.34   
1      2023-08-25 09:52:04.890050816     10.42.0.34     10.42.0.111   
2      2023-08-25 09:52:04.890074880     10.42.0.34     10.42.0.111   
3      2023-08-25 09:52:04.890078976     10.42.0.34     10.42.0.111   
4      2023-08-25 09:52:04.890081792     10.42.0.34     10.42.0.111   
...                              ...            ...             ...   
529951 2023-08-25 10:18:47.776020992    10.42.0.111      10.42.0.34   
529952 2023-08-25 10:18:47.783264000  57.128.101.93     10.42.0.111   
529953 2023-08-25 10:18:47.811702016    10.42.0.111     224.0.0.251   
529954 2023-08-25 10:18:47.811779072            NaN             NaN   
529955 2023-08-25 10:18:47.812879104    10.42.0.111  185.125.190.18   

        source_port  destination_port  protocol  packet_size  \
0           14550.0           53701.0      17.0           65   
1           53701.0

code for calculatting using 10 sec window

In [ ]:
import pandas as pd
import csv

# Sort data by timestamp
data = data.sort_values(by='adjusted_timestamp')

# Define window size and frequency
window_size = pd.Timedelta(seconds=0.25)
window_freq = pd.Timedelta(seconds=0.25)

# Group data by timestamp and resample with the window frequency
group_data = data.set_index('adjusted_timestamp').resample(window_freq)

# Create a list to store packet information for each window
packet_classification = []

# Iterate through the resampled time windows
for window_start, window_data in group_data:
    if not window_data.empty:
        window_packet_sizes = window_data['packet_size']
        inter_arrival_times = [(window_data.index[i] - window_data.index[i - 1]).total_seconds() for i in range(1, len(window_data))]

        total_packets = len(window_packet_sizes)
        total_size = window_packet_sizes.sum()
        mean_size = total_size / total_packets if total_packets > 0 else 0
        mean_inter_arrival = sum(inter_arrival_times) / len(inter_arrival_times) if len(inter_arrival_times) > 0 else 0

        # Append packet information to the list
        packet_classification.append([window_start, total_packets, total_size, mean_size, mean_inter_arrival])

# Define the output CSV file name
output_csv_filename = 'norm_silv(0.25s).csv'

# Write the packet information to the CSV file
with open(output_csv_filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)

    # Write the header row
    csv_writer.writerow(['Window start', 'Total packets', 'Total size', 'Mean size', 'Mean inter-arrival'])

    # Write the packet information rows
    for packet_info in packet_classification:
        csv_writer.writerow(packet_info)

print("Results written to", output_csv_filename)




Results written to norm_silv(0.25s).csv


In [ ]:
data=pd.read_csv('/content/packet_info(1s).csv')
data.head()

,Window start,Total packets,Total size,Mean size,Mean inter-arrival
0,0 days 00:00:01,56,3702,66.107143,0.018181
1,0 days 00:00:02,63,4392,69.714286,0.015475
2,0 days 00:00:03,56,3684,65.785714,0.017447
3,0 days 00:00:04,56,3732,66.642857,0.017505
4,0 days 00:00:05,55,3665,66.636364,0.017850


In [ ]:
pip install pyshark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.1 MB/s eta 0:00:00


In [ ]:
pip install scapy

In [ ]:
from scapy.all import *
import csv

def pcap_to_csv(pcap_filename, csv_filename):
    packets = rdpcap(pcap_filename)

    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ['timestamp', 'source_ip', 'destination_ip', 'source_port', 'destination_port', 'protocol', 'packet_size']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()

        for pkt in packets:
            timestamp = pkt.time
            source_ip = pkt[IP].src if IP in pkt else None
            destination_ip = pkt[IP].dst if IP in pkt else None
            source_port = pkt[IP].sport if IP in pkt and hasattr(pkt[IP], 'sport') else None
            destination_port = pkt[IP].dport if IP in pkt and hasattr(pkt[IP], 'dport') else None
            protocol = pkt[IP].proto if IP in pkt else None
            packet_size = len(pkt)

            writer.writerow({
                'timestamp': timestamp,
                'source_ip': source_ip,
                'destination_ip': destination_ip,
                'source_port': source_port,
                'destination_port': destination_port,
                'protocol': protocol,
                'packet_size': packet_size
            })

# Specify input PCAP file and output CSV file
input_pcap_filename = ''
output_csv_filename = 'big_ddos(0.25s).csv'

pcap_to_csv(input_pcap_filename, output_csv_filename)
print("PCAP to CSV conversion complete. Results written to", output_csv_filename)



PCAP to CSV conversion complete. Results written to big_ddos(0.25s).csv


code for adjust timestamp

In [ ]:
import pandas as pd

# Sample data with Unix epoch timestamps
data = pd.read_csv('/content/traffic_data (1).csv')

# Convert Unix epoch timestamps to pandas Timestamp objects
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')

# Find the minimum Unix epoch timestamp
min_timestamp = data['timestamp'].min()

# Subtract the minimum timestamp to shift the time scale
data['adjusted_timestamp'] = (data['timestamp'] - min_timestamp) + pd.Timedelta(seconds=1)

print(data)

                           timestamp        source_ip destination_ip  \
0      2023-08-20 06:32:43.277779200      10.42.0.111     10.42.0.34   
1      2023-08-20 06:32:43.277807872      10.42.0.111     10.42.0.34   
2      2023-08-20 06:32:43.317961984      10.42.0.111     10.42.0.34   
3      2023-08-20 06:32:43.317982976      10.42.0.111     10.42.0.34   
4      2023-08-20 06:32:43.357772032      10.42.0.111     10.42.0.34   
...                              ...              ...            ...   
114144 2023-08-20 06:34:11.830530816   123.132.91.244    10.42.0.111   
114145 2023-08-20 06:34:11.830534912   174.223.68.197    10.42.0.111   
114146 2023-08-20 06:34:11.831715072  249.135.138.233    10.42.0.111   
114147 2023-08-20 06:34:11.831727872   123.37.126.242    10.42.0.111   
114148 2023-08-20 06:34:11.831731968  255.204.107.132     10.42.0.11   

        source_port  destination_port  protocol  packet_size  \
0           14550.0           44840.0      17.0         65.0   
1      

code for packet_info normal

In [ ]:
import pandas as pd
import csv

# Sort data by timestamp
data = data.sort_values(by='adjusted_timestamp')

# Define window size and frequency
window_size = pd.Timedelta(seconds=0.25)
window_freq = pd.Timedelta(seconds=0.25)

# Group data by timestamp and resample with the window frequency
group_data = data.set_index('adjusted_timestamp').resample(window_freq)

# Create a list to store packet information for each window
packet_classification = []

# Iterate through the resampled time windows
for window_start, window_data in group_data:
    if not window_data.empty:
        window_packet_sizes = window_data['packet_size']
        inter_arrival_times = [(window_data.index[i] - window_data.index[i - 1]).total_seconds() for i in range(1, len(window_data))]

        total_packets = len(window_packet_sizes)#no of packets in window
        total_size = window_packet_sizes.sum()#size of all packets in window
        mean_size = total_size / total_packets if total_packets > 0 else 0
        mean_inter_arrival = sum(inter_arrival_times) / len(inter_arrival_times) if len(inter_arrival_times) > 0 else 0

        # Append packet information to the list
        packet_classification.append([window_start, total_packets, total_size, mean_size, mean_inter_arrival])

# Define the output CSV file name
output_csv_filename = 'black.csv'

# Write the packet information to the CSV file
with open(output_csv_filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)

    # Write the header row
    csv_writer.writerow(['Window start', 'Total packets', 'Total size', 'Mean size', 'Mean inter-arrival'])

    # Write the packet information rows
    for packet_info in packet_classification:
        csv_writer.writerow(packet_info)

print("Results written to", output_csv_filename)



Results written to black.csv


In [ ]:
import pandas as pd

# Read the first CSV file
csv_file1 = '/content/packet_info (2).csv'  # Replace with your file name
df1 = pd.read_csv(csv_file1)

# Read the second CSV file
csv_file2 = '/content/packet_info_normal (1).csv'  # Replace with your file name
df2 = pd.read_csv(csv_file2)

# Merge the two DataFrames based on common columns
merged_df = pd.merge(df1, df2, on=['Total packets', 'Total size', 'Mean size', 'Mean inter-arrival'])

# Save the merged DataFrame to a new CSV file
merged_csv_file = 'merged_file.csv'
merged_df.to_csv(merged_csv_file, index=False)

print(f"Merged data saved to {merged_csv_file}")


Merged data saved to merged_file.csv


In [ ]:
# Support Vector Machine (SVM)

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import ConfusionMatrixDisplay


# Importing the dataset
dataset = pd.read_csv('merged_file.csv')

# Separate features and target variable
X = dataset.drop('Scenario', axis=1).values
y = dataset['Scenario'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# Initialize the scalers
scaler_total_packets = StandardScaler()
scaler_total_size = StandardScaler()
scaler_mean_size = StandardScaler()
scaler_mean_interarrival = StandardScaler()

# Normalize 'Total packets'
X_train[:, 0] = scaler_total_packets.fit_transform(X_train[:, 0].reshape(-1, 1)).flatten()
X_test[:, 0] = scaler_total_packets.transform(X_test[:, 0].reshape(-1, 1)).flatten()

# Normalize 'Total size'
X_train[:, 1] = scaler_total_size.fit_transform(X_train[:, 1].reshape(-1, 1)).flatten()
X_test[:, 1] = scaler_total_size.transform(X_test[:, 1].reshape(-1, 1)).flatten()

# Normalize 'Mean size'
X_train[:, 2] = scaler_mean_size.fit_transform(X_train[:, 2].reshape(-1, 1)).flatten()
X_test[:, 2] = scaler_mean_size.transform(X_test[:, 2].reshape(-1, 1)).flatten()

# Normalize 'Mean inter-arrival'
X_train[:, 3] = scaler_mean_interarrival.fit_transform(X_train[:, 3].reshape(-1, 1)).flatten()
X_test[:, 3] = scaler_mean_interarrival.transform(X_test[:, 3].reshape(-1, 1)).flatten()
#print(X_train)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

classifier = SVC(kernel='linear', random_state=0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

# Calculating accuracy and confusion matrix
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

report = classification_report(y_test, y_pred, target_names=['Normal', 'Anomaly'])
print("Classification Report:\n", report)

#Visualizing the Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Oranges", cbar=False)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.savefig('Confusion Matrix.png', dpi=600)  # Save confusion matrix as PNG
plt.show()


# Display confusion matrix from estimator
target_names = ['Normal', 'Anomaly']
ConfusionMatrixDisplay.from_estimator(classifier, X_test, y_test, display_labels=target_names, normalize=None, cmap="OrRd")
plt.title('Testing Confusion Matrix SVM')
plt.savefig('Testing Confusion Matrix SVM.png', dpi=600)
plt.show()

#Calculate runtime
start_time = time.time()  # Start time for runtime measurement
end_time = time.time()
total_runtime = end_time - start_time

print("Total runtime:", total_runtime, "seconds")